# Cleaning up messy data

## Install

In [1]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# !pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

Name: syft
Version: 0.8.1b3
Summary: Perform numpy-like analysis on data that remains in someone elses server
Home-page: https://openmined.github.io/PySyft/
Author: OpenMined
Author-email: info@openmined.org
License: Apache-2.0
Location: /Users/theresa/opt/anaconda3/envs/OpenMined/lib/python3.9/site-packages
Requires: astunparse, bcrypt, fastapi, flax, forbiddenfruit, gevent, gipc, hagrid, jax, jaxlib, loguru, networkx, numpy, opendp, opentelemetry-api, opentelemetry-exporter-jaeger, opentelemetry-instrumentation, opentelemetry-instrumentation-requests, opentelemetry-sdk, packaging, pandas, pyarrow, pycapnp, pydantic, pymongo, pynacl, pyzmq, redis, requests, RestrictedPython, result, sherlock, tqdm, typeguard, typing-extensions, uvicorn
Required-by: 


In [2]:
import syft as sy
sy.requires(SYFT_VERSION)

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /


Failed to find type for key: value in <enum '_NoDefault'>
Failed to find type for key: __origin__ in <class 'typing._UnionGenericAlias'>
Failed to find type for key: _value in <class 'result.result.Err'>
Failed to find type for key: __origin__ in <class 'typing._UnionGenericAlias'>
Failed to find type for key: _monitor_socket in <class 'zmq.green.core._Socket'>
Failed to find type for key: value in <enum 'NodeType'>
Failed to find type for key: value in <enum 'ServiceRole'>
Failed to find type for key: value in <enum 'ActionPermission'>
Failed to find type for key: store_config in <class 'syft.store.document_store.StorePartition'>
Failed to find type for key: path in <class 'syft.types.grid_url.GridURL'>
Failed to find type for key: value in <enum 'CMPCRUDPermission'>
Failed to find type for key: value in <enum '_ParameterKind'>
Failed to find type for key: value in <enum 'VPNRoutes'>
Failed to find type for key: value in <enum 'CommandStatus'>
Failed to find type for key: value in <en

In [3]:
node = sy.orchestra.launch(name="pandas-test-domain-1", reset=True)

SQLite Store Path:
!open file:///var/folders/f1/h55w4kj150x0s8c3jwhkkygw0000gn/T/281e55cb3425460e864682cfb4672fcb.sqlite



# Data owner: Upload data

In [4]:
root_domain_client = node.login(email="info@openmined.org", password="changethis")

## Load data

In [5]:
# The usual preamble
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from syft.service.project.project import Project
from syft.util.util import autocache, PANDAS_DATA

# Make the graphs a bit prettier, and bigger
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['font.family'] = 'sans-serif'

# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

One of the main problems with messy data is: how do you know if it's messy or not?

We're going to use the NYC 311 service request dataset again here, since it's big and a bit unwieldy.

## Create mock data

In [6]:
service_requests = pd.read_csv(autocache(f"{PANDAS_DATA}/311-service-requests.csv"), dtype='unicode')

In [7]:
service_requests.head(100)

Unique Key            Created Date             Closed Date Agency                              Agency Name           Complaint Type                    Descriptor        Location Type Incident Zip       Incident Address       Street Name       Cross Street 1                    Cross Street 2 Intersection Street 1 Intersection Street 2  Address Type           City Landmark Facility Type    Status                Due Date Resolution Action Updated Date Community Board    Borough X Coordinate (State Plane) Y Coordinate (State Plane) Park Facility Name Park Borough  School Name School Number School Region  School Code School Phone Number School Address  School City School State   School Zip School Not Found School or Citywide Complaint Vehicle Type Taxi Company Borough Taxi Pick Up Location Bridge Highway Name Bridge Highway Direction Road Ramp Bridge Highway Segment Garage Lot Name Ferry Direction Ferry Terminal Name            Latitude           Longitude                                  Location
0    26589651  10/31/2013 02:08:41 AM                     NaN   NYPD          New York City Police Department  Noise - Street/Sidewalk                  Loud Talking      Street/Sidewalk        11432       90-03 169 STREET        169 STREET            90 AVENUE                         91 AVENUE                   NaN                   NaN       ADDRESS        JAMAICA      NaN      Precinct  Assigned  10/31/2013 10:08:41 AM         10/31/2013 02:35:17 AM       12 QUEENS     QUEENS                    1042027                     197389        Unspecified       QUEENS  Unspecified   Unspecified   Unspecified  Unspecified         Unspecified    Unspecified  Unspecified  Unspecified  Unspecified                N                          NaN          NaN                  NaN                   NaN                 NaN                      NaN       NaN                    NaN             NaN             NaN                 NaN   40.70827532593202  -73.79160395779721   (40.70827532593202, -73.79160395779721)
1    26593698  10/31/2013 02:01:04 AM                     NaN   NYPD          New York City Police Department          Illegal Parking  Commercial Overnight Parking      Street/Sidewalk        11378              58 AVENUE         58 AVENUE             58 PLACE                         59 STREET                   NaN                   NaN     BLOCKFACE        MASPETH      NaN      Precinct      Open  10/31/2013 10:01:04 AM                            NaN       05 QUEENS     QUEENS                    1009349                     201984        Unspecified       QUEENS  Unspecified   Unspecified   Unspecified  Unspecified         Unspecified    Unspecified  Unspecified  Unspecified  Unspecified                N                          NaN          NaN                  NaN                   NaN                 NaN                      NaN       NaN                    NaN             NaN             NaN                 NaN  40.721040535628305  -73.90945306791765  (40.721040535628305, -73.90945306791765)
2    26594139  10/31/2013 02:00:24 AM  10/31/2013 02:40:32 AM   NYPD          New York City Police Department       Noise - Commercial              Loud Music/Party  Club/Bar/Restaurant        10032          4060 BROADWAY          BROADWAY      WEST 171 STREET                   WEST 172 STREET                   NaN                   NaN       ADDRESS       NEW YORK      NaN      Precinct    Closed  10/31/2013 10:00:24 AM         10/31/2013 02:39:42 AM    12 MANHATTAN  MANHATTAN                    1001088                     246531        Unspecified    MANHATTAN  Unspecified   Unspecified   Unspecified  Unspecified         Unspecified    Unspecified  Unspecified  Unspecified  Unspecified                N                          NaN          NaN                  NaN                   NaN                 NaN                      NaN       NaN                    NaN             NaN             NaN                 NaN   40.84332975466513  -73.93914371913482  

In [8]:
rows_with_dashes = service_requests['Incident Zip'].str.contains('-').fillna(False)
service_requests[rows_with_dashes]

Unique Key            Created Date             Closed Date Agency                     Agency Name      Complaint Type         Descriptor Location Type Incident Zip        Incident Address         Street Name Cross Street 1 Cross Street 2 Intersection Street 1 Intersection Street 2 Address Type        City Landmark Facility Type    Status                Due Date Resolution Action Updated Date Community Board      Borough X Coordinate (State Plane) Y Coordinate (State Plane) Park Facility Name Park Borough  School Name School Number School Region  School Code School Phone Number School Address  School City School State   School Zip School Not Found School or Citywide Complaint Vehicle Type Taxi Company Borough Taxi Pick Up Location Bridge Highway Name Bridge Highway Direction Road Ramp Bridge Highway Segment Garage Lot Name Ferry Direction Ferry Terminal Name Latitude Longitude Location
29136   26550551  10/24/2013 06:16:34 PM                     NaN    DCA  Department of Consumer Affairs  Consumer Complaint  False Advertising           NaN   77092-2016   2700 EAST SELTICE WAY    EAST SELTICE WAY            NaN            NaN                   NaN                   NaN          NaN     HOUSTON      NaN           NaN  Assigned  11/13/2013 11:15:20 AM         10/29/2013 11:16:16 AM   0 Unspecified  Unspecified                        NaN                        NaN        Unspecified  Unspecified  Unspecified   Unspecified   Unspecified  Unspecified         Unspecified    Unspecified  Unspecified  Unspecified  Unspecified                N                          NaN          NaN                  NaN                   NaN                 NaN                      NaN       NaN                    NaN             NaN             NaN                 NaN      NaN       NaN      NaN
30939   26548831  10/24/2013 09:35:10 AM                     NaN    DCA  Department of Consumer Affairs  Consumer Complaint         Harassment           NaN   55164-0737          P.O. BOX 64437               64437            NaN            NaN                   NaN                   NaN          NaN    ST. PAUL      NaN           NaN  Assigned  11/13/2013 02:30:21 PM         10/29/2013 02:31:06 PM   0 Unspecified  Unspecified                        NaN                        NaN        Unspecified  Unspecified  Unspecified   Unspecified   Unspecified  Unspecified         Unspecified    Unspecified  Unspecified  Unspecified  Unspecified                N                          NaN          NaN                  NaN                   NaN                 NaN                      NaN       NaN                    NaN             NaN             NaN                 NaN      NaN       NaN      NaN
70539   26488417  10/15/2013 03:40:33 PM                     NaN    TLC   Taxi and Limousine Commission      Taxi Complaint   Driver Complaint        Street   11549-3650  365 HOFSTRA UNIVERSITY  HOFSTRA UNIVERSITY            NaN            NaN                   NaN                   NaN          NaN    HEMSTEAD      NaN           NaN  Assigned  11/30/2013 01:20:33 PM         10/16/2013 01:21:39 PM   0 Unspecified  Unspecified                        NaN                        NaN        Unspecified  Unspecified  Unspecified   Unspecified   Unspecified  Unspecified         Unspecified    Unspecified  Unspecified  Unspecified  Unspecified                N                          NaN          NaN                  NaN    La Guardia Airport                 NaN                      NaN       NaN                    NaN             NaN             NaN                 NaN      NaN       NaN      NaN
85821   26468296  10/10/2013 12:36:43 PM  10/26/2013 01:07:07 AM    DCA  Department of Consumer Affairs  Consumer Complaint      Debt Not Owed           NaN   29616-0759            PO BOX 25759           BOX 25759            NaN            NaN                   NaN                   NaN          NaN  GREENVILLE      NaN           NaN    Closed  10/26/2013 09:20:28 AM         10/26/2013 0

In [9]:
import random
def get_unique_key():
    return random.randint(0,1000000)
    
def get_mock_location():
    return random.uniform(-90, 90)

def get_zip_code():
    zip = random.randint(10000,11000)
    if zip > 10990:
        zip = str(zip) + '-1234'
    return str(zip)

def get_mock_row(i):
    res = dict()
    for k, function in mock_functions.items():
        res[k] = function()
    return res
    

In [10]:
# make mock as close to the original data as possible!!
# TODO: Make it the same as the OG dataframe
mock_functions = {
    "Unique Key": lambda: get_unique_key(),
    'Longitude': lambda: random.uniform(-90, 90),
    'Latitude': lambda: random.uniform(-90, 90),
    'Incident Zip': lambda: get_zip_code(),
    'City': lambda: random.choice(["BROOKLYN", "NEW YORK", "BRONX"])
}

In [11]:
mock = pd.DataFrame(data=[get_mock_row(i) for i in range(len(service_requests))],
                    columns=service_requests.columns)

In [12]:
mock

Unique Key  Created Date  Closed Date  Agency  Agency Name  Complaint Type  Descriptor  Location Type Incident Zip  Incident Address  Street Name  Cross Street 1  Cross Street 2  Intersection Street 1  Intersection Street 2  Address Type      City  Landmark  Facility Type  Status  Due Date  Resolution Action Updated Date  Community Board  Borough  X Coordinate (State Plane)  Y Coordinate (State Plane)  Park Facility Name  Park Borough  School Name  School Number  School Region  School Code  School Phone Number  School Address  School City  School State  School Zip  School Not Found  School or Citywide Complaint  Vehicle Type  Taxi Company Borough  Taxi Pick Up Location  Bridge Highway Name  Bridge Highway Direction  Road Ramp  Bridge Highway Segment  Garage Lot Name  Ferry Direction  Ferry Terminal Name   Latitude  Longitude  Location
0           292141           NaN          NaN     NaN          NaN             NaN         NaN            NaN        10037               NaN          NaN             NaN             NaN                    NaN                    NaN           NaN  NEW YORK       NaN            NaN     NaN       NaN                             NaN              NaN      NaN                         NaN                         NaN                 NaN           NaN          NaN            NaN            NaN          NaN                  NaN             NaN          NaN           NaN         NaN               NaN                           NaN           NaN                   NaN                    NaN                  NaN                       NaN        NaN                     NaN              NaN              NaN                  NaN -66.524186 -84.697449       NaN
1            77040           NaN          NaN     NaN          NaN             NaN         NaN            NaN        10011               NaN          NaN             NaN             NaN                    NaN                    NaN           NaN     BRONX       NaN            NaN     NaN       NaN                             NaN              NaN      NaN                         NaN                         NaN                 NaN           NaN          NaN            NaN            NaN          NaN                  NaN             NaN          NaN           NaN         NaN               NaN                           NaN           NaN                   NaN                    NaN                  NaN                       NaN        NaN                     NaN              NaN              NaN                  NaN  85.876492  -2.093488       NaN
2           191215           NaN          NaN     NaN          NaN             NaN         NaN            NaN        10701               NaN          NaN             NaN             NaN                    NaN                    NaN           NaN     BRONX       NaN            NaN     NaN       NaN                             NaN              NaN      NaN                         NaN                         NaN                 NaN           NaN          NaN            NaN            NaN          NaN                  NaN             NaN          NaN           NaN         NaN               NaN                           NaN           NaN                   NaN                    NaN                  NaN                       NaN        NaN                     NaN              NaN              NaN                  NaN  52.265827  50.406626       NaN
3           454627           NaN          NaN     NaN          NaN             NaN         NaN            NaN        10099               NaN          NaN             NaN             NaN                    NaN                    NaN           NaN  NEW YORK       NaN            NaN     NaN       NaN                             NaN              NaN      NaN                         NaN                         NaN                 NaN           NaN          NaN            NaN            NaN          NaN                  NaN             NaN          NaN           NaN         NaN          

Upload the data

In [13]:
dataset = sy.Dataset(name="test", asset_list=[sy.Asset(name=f"service_requests",
                                                       data=service_requests[:1000],
                                                       mock=mock[:1000], # TODO: remove :1000
                                                       mock_is_real=False)])
root_domain_client.upload_dataset(dataset)


100%|██████████| 1/1 [00:00<00:00,  5.24it/s]


Uploading: service_requests


<class 'syft.service.response.SyftSuccess'>: Dataset Added

## Create user account

In [14]:
user = root_domain_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")
# todo: give user data scientist role
guest_domain_client = node.client
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

# Data scientist: create result pointer

In [15]:
import numpy as np
import pandas as pd

## Summary
By the end of this chapter, we're going to have downloaded all of Canada's weather data for 2012, and saved it to a CSV.

We'll do this by downloading it one month at a time, and then combining all the months together.

## Get mocks

In [16]:
ds = guest_domain_client.datasets[0]

In [17]:
asset = ds.assets[0]

In [18]:
requests = asset.mock

## How do we know if it's messy?
We're going to look at a few columns here. I know already that there are some problems with the zip code, so let's look at that first.

To get a sense for whether a column has problems, I usually use `.unique()` to look at all its values. If it's a numeric column, I'll instead plot a histogram to get a sense of the distribution.

When we look at the unique values in "Incident Zip", it quickly becomes clear that this is a mess.

Some of the problems:

- Some have been parsed as strings, and some as floats
- There are `nans`
- Some of the zip codes are `29616-0759` or `83`
- There are some N/A values that pandas didn't recognize, like 'N/A' and 'NO CLUE'

What we can do:

- Normalize 'N/A' and 'NO CLUE' into regular nan values
- Look at what's up with the 83, and decide what to do
- Make everything strings

In [19]:
res = requests['Incident Zip'].unique()

## Fixing the nan values and string/float confusion
We can pass a na_values option to pd.read_csv to clean this up a little bit. We can also specify that the type of Incident Zip is a string, not a float.



In [20]:
na_values = ['NO CLUE', 'N/A', '0']
requests.replace(na_values, np.NaN);

In [21]:
requests['Incident Zip'].unique()

```python
TwinPointer(Mock)
```
array(['10037', '10011', '10701', '10099', '10389', '10195', '10769',
       '10709', '10872', '10008', '10612', '10712', '10050', '10319',
       '10076', '10674', '10908', '10207', '10147', '10802', '10120',
       '10376', '10027', '10000', '10507', '10620', '10269', '10649',
       '10682', '10696', '10819', '10103', '10675', '10482', '10594',
       '10998-1234', '10854', '10619', '10573', '10814', '10452', '10181',
       '10089', '10879', '10420', '10402', '10809', '10941', '10168',
       '10925', '10997-1234', '10508', '10725', '10486', '10688', '10078',
       '10023', '10077', '10722', '10758', '10747', '10918', '10484',
       '10129', '10993-1234', '10014', '10866', '10940', '10711', '10163',
       '10782', '10426', '10344', '10791', '10892', '10105', '10466',
       '10666', '10909', '10497', '10989', '10057', '10408', '10272',
       '10144', '10862', '10912', '10406', '10331', '10357', '10398',
       '10833', '10362', '10622', '10877', '10134', '10471', '10766',
       '10763', '10639', '10298', '10536', '10164', '10038', '10087',
       '10431', '10653', '10962', '10829', '10204', '10019', '10656',
       '10844', '10145', '10916', '10001', '10066', '10029', '10836',
       '10102', '10929', '10652', '10243', '10642', '10774', '10046',
       '10292', '10209', '10806', '10838', '10807', '10149', '10045',
       '10772', '10804', '10113', '10043', '10679', '10531', '10979',
       '10202', '10974', '10553', '10724', '10092', '10729', '10692',
       '10808', '10392', '10499', '10192', '10302', '10260', '10034',
       '10794', '10485', '10852', '10615', '10832', '10950', '10767',
       '10430', '10516', '10241', '10154', '10861', '10927', '10403',
       '10650', '10647', '10210', '10784', '10847', '10936', '10253',
       '10826', '10588', '10311', '10441', '10451', '10173', '10907',
       '10194', '10282', '10340', '10032', '10437', '10009', '10067',
       '10917', '10265', '10965', '10972', '10064', '10114', '10641',
       '10198', '10775', '10714', '10676', '10853', '10455', '10756',
       '10532', '10995-1234', '10188', '10257', '10438', '10977', '10870',
       '10132', '10271', '10180', '10752', '10219', '10693', '10932',
       '10582', '10822', '10953', '10668', '10737', '10745', '10462',
       '10458', '10562', '10121', '10719', '10122', '10473', '10459',
       '10310', '10290', '10117', '10777', '10789', '10284', '10738',
       '10218', '10554', '10104', '10187', '10704', '10888', '10463',
       '10303', '10016', '10135', '10382', '10434', '10768', '10428',
       '10317', '10878', '10818', '10030', '10842', '10760', '10889',
       '10960', '10602', '10762', '10616', '10894', '10111', '10800',
       '10380', '10494', '10444', '10942', '10565', '10864', '10230',
       '10404', '10312', '10583', '10748', '10515', '10300', '10239',
       '10213', '10283', '10096', '10320', '10176', '10659', '10886',
       '10183', '10623', '10123', '10810', '10423', '10933', '10017',
       '10448', '10366', '10259', '10435', '10443', '10206', '10280',
       '10231', '10155', '10351', '10359', '10232', '10150', '10005',
       '10868', '10369', '10744', '10543', '10655', '10954', '10094',
       '10885', '10992-1234', '10297', '10736', '10279', '10397', '10461',
       '10294', '10322', '10073', '10372', '10773', '10813', '10415',
       '10413', '10728', '10022', '10839', '10590', '10254', '10963',
       '10955', '10891', '10186', '10054', '10566', '10860', '10370',
       '10946', '10309', '10906', '10490', '10267', '10801', '10411',
       '10939', '10528', '10502', '10042', '10429', '10947', '10727',
       '10041', '10967', '10004', '10033', '10140', '10926', '10112',
       '10307', '10235', '10162', '10990', '10610', '10673', '10923',
       '10125', '10228', '10781', '10208', '10683', '10336', '10385',
       '10250', '10710', '10258', '10447', '10091', '10399', '10015',
       '10240', '10581', '10433', '10255', '10360', '10069', '10238',
       '10578', '10700', '10048', '10550', '10637', '10897', '10086',
       '10146', '10301', '10934', '10851', '10887', '10075', '10634',
       '10171', '10107', '10924', '10495', '10548', '10734', '10827',
       '10787', '10080', '10805', '10949', '10614', '10287', '10837',
       '10249', '10568', '10445', '10957', '10834', '10074', '10483',
       '10667', '10975', '10165', '10044', '10600', '10236', '10764',
       '10400', '10215', '10523', '10386', '10013', '10723', '10503',
       '10607', '10824', '10721', '10720', '10884', '10118', '10914',
       '10717', '10116', '10024', '10109', '10002', '10056', '10811',
       '10489', '10358', '10095', '10031', '10670', '10893', '10436',
       '10874', '10115', '10983', '10371', '10625', '10935', '10797',
       '10025', '10790', '10759', '10060', '10750', '10110', '10101',
       '10969', '10713', '10409', '10323', '10518', '10421', '10052',
       '10976', '10393', '10161', '10053', '10903', '10512', '10263',
       '10978', '10741', '10569', '10793', '10702', '10491', '10456',
       '10678', '10922', '10093', '10742', '10665', '10201', '10627',
       '10252', '10460', '10831', '10363', '10895', '10973', '10496',
       '10305', '10142', '10055', '10618', '10467', '10651', '10221',
       '10106', '10855', '10432', '10945', '10743', '10304', '10396',
       '10355', '10449', '10159', '10273', '10079', '10857', '10346',
       '10823', '10921', '10214', '10178', '10266', '10900', '10996-1234',
       '10384', '10544', '10244', '10148', '10783', '10314', '10136',
       '10648', '10157', '10337', '10407', '10686', '10645', '10931',
       '10487', '10606', '10422', '10158', '10217', '10964', '10952',
       '10890', '10081', '10991-1234', '10796', '10640', '10151', '10504',
       '10551', '10587', '10731', '10685', '10930', '10328', '10630',
       '10687', '10167', '10604', '10020', '10216', '10182', '10968',
       '10980', '10788', '10883', '10224', '10570', '10795', '10414',
       '10262', '10334', '10792', '10876', '10418', '10869', '10356',
       '10251', '10911', '10849', '10603', '10785', '10680', '10324',
       '10705', '10803', '10374', '10517', '10417', '10133', '10410',
       '10419', '10276', '10179', '10881', '10169', '10465', '10858',
       '10329', '10335', '10740', '10242', '10083', '10577', '10040',
       '10394', '10799', '10354', '10613', '10644', '10987', '10735',
       '10457', '10593', '10425', '10367', '10601', '10985', '10330',
       '10036', '10012', '10139', '10718'], dtype=object)

## What's up with the dashes?

In [22]:
rows_with_dashes = requests['Incident Zip'].str.contains('-').fillna(False)
len(requests[rows_with_dashes])

11

In [23]:
requests[rows_with_dashes]

TwinPointer(Mock):
     Unique Key  Created Date  Closed Date  Agency  Agency Name  Complaint Type  Descriptor  Location Type Incident Zip  Incident Address  Street Name  Cross Street 1  Cross Street 2  Intersection Street 1  Intersection Street 2  Address Type      City  Landmark  Facility Type  Status  Due Date  Resolution Action Updated Date  Community Board  Borough  X Coordinate (State Plane)  Y Coordinate (State Plane)  Park Facility Name  Park Borough  School Name  School Number  School Region  School Code  School Phone Number  School Address  School City  School State  School Zip  School Not Found  School or Citywide Complaint  Vehicle Type  Taxi Company Borough  Taxi Pick Up Location  Bridge Highway Name  Bridge Highway Direction  Road Ramp  Bridge Highway Segment  Garage Lot Name  Ferry Direction  Ferry Terminal Name   Latitude  Longitude  Location
36       689137           NaN          NaN     NaN          NaN             NaN         NaN            NaN   10998-1234               NaN          NaN             NaN             NaN                    NaN                    NaN           NaN     BRONX       NaN            NaN     NaN       NaN                             NaN              NaN      NaN                         NaN                         NaN                 NaN           NaN          NaN            NaN            NaN          NaN                  NaN             NaN          NaN           NaN         NaN               NaN                           NaN           NaN                   NaN                    NaN                  NaN                       NaN        NaN                     NaN              NaN              NaN                  NaN  -9.608005  78.082348       NaN
52       158430           NaN          NaN     NaN          NaN             NaN         NaN            NaN   10997-1234               NaN          NaN             NaN             NaN                    NaN                    NaN           NaN  NEW YORK       NaN            NaN     NaN       NaN                             NaN              NaN      NaN                         NaN                         NaN                 NaN           NaN          NaN            NaN            NaN          NaN                  NaN             NaN          NaN           NaN         NaN               NaN                           NaN           NaN                   NaN                    NaN                  NaN                       NaN        NaN                     NaN              NaN              NaN                  NaN  15.799152 -74.214252       NaN
67       973040           NaN          NaN     NaN          NaN             NaN         NaN            NaN   10993-1234               NaN          NaN             NaN             NaN                    NaN                    NaN           NaN  BROOKLYN       NaN            NaN     NaN       NaN                             NaN              NaN      NaN                         NaN                         NaN                 NaN           NaN          NaN            NaN            NaN          NaN                  NaN             NaN          NaN           NaN         NaN               NaN                           NaN           NaN                   NaN                    NaN                  NaN                       NaN        NaN                     NaN              NaN              NaN                  NaN  25.067521  10.199095       NaN
179      779141           NaN          NaN     NaN          NaN             NaN         NaN            NaN   10993-1234               NaN          NaN             NaN             NaN                    NaN                    NaN           NaN  BROOKLYN       NaN            NaN     NaN       NaN                             NaN              NaN      NaN                         NaN                         NaN                 NaN           NaN          NaN            NaN            NaN          NaN                  NaN             NaN          NaN           NaN         N

I thought these were missing data and originally deleted them like this:

`requests['Incident Zip'][rows_with_dashes] = np.nan`

But then my friend Dave pointed out that 9-digit zip codes are normal. Let's look at all the zip codes with more than 5 digits, make sure they're okay, and then truncate them.

In [24]:
long_zip_codes = requests['Incident Zip'].str.len() > 5
requests['Incident Zip'][long_zip_codes].unique()

```python
TwinPointer(Mock)
```
array(['10998-1234', '10997-1234', '10993-1234', '10995-1234',
       '10992-1234', '10996-1234', '10991-1234'], dtype=object)

Those all look okay to truncate to me.

In [25]:
requests['Incident Zip'] = requests['Incident Zip'].str.slice(0, 5)

Done.

Earlier I thought 00083 was a broken zip code, but turns out Central Park's zip code 00083! Shows what I know. I'm still concerned about the 00000 zip codes, though: let's look at that.

In [26]:
requests[requests['Incident Zip'] == '00000'] 

TwinPointer(Mock):
Empty DataFrame
Columns: [Unique Key, Created Date, Closed Date, Agency, Agency Name, Complaint Type, Descriptor, Location Type, Incident Zip, Incident Address, Street Name, Cross Street 1, Cross Street 2, Intersection Street 1, Intersection Street 2, Address Type, City, Landmark, Facility Type, Status, Due Date, Resolution Action Updated Date, Community Board, Borough, X Coordinate (State Plane), Y Coordinate (State Plane), Park Facility Name, Park Borough, School Name, School Number, School Region, School Code, School Phone Number, School Address, School City, School State, School Zip, School Not Found, School or Citywide Complaint, Vehicle Type, Taxi Company Borough, Taxi Pick Up Location, Bridge Highway Name, Bridge Highway Direction, Road Ramp, Bridge Highway Segment, Garage Lot Name, Ferry Direction, Ferry Terminal Name, Latitude, Longitude, Location]
Index: []

This looks bad to me. Let's set these to nan.

**note**: this uses `requests[zero_zips, 'Incident Zip']` instead of `requests.loc[zero_zips, 'Incident Zip']`. This is because of a limitation of the current version of syft. If we have a pointer and we do ptr.a, we actually get a copy of a. If we then update a we are actually updating the copy, and not the ptr. This is a problem for `DataFrame.loc[x]`.

In [27]:
zero_zips = requests['Incident Zip'] == '00000'
# this is currently not possible
# requests.loc[zero_zips, 'Incident Zip'] = np.nan

# this is the workaround
incident_zips = requests['Incident Zip']
incident_zips[zero_zips] = np.nan

requests['Incident Zip'] = incident_zips

/Users/theresa/opt/anaconda3/envs/OpenMined/lib/python3.9/site-packages/syft/service/action/action_object.py:1120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  local_func(*original_args, **original_kwargs)


Great. Let's see where we are now:

**This is supposed to error**

In [28]:
# this is supposed to error
unique_zips = requests['Incident Zip'].unique()
unique_zips.sort()
unique_zips

send_action_side_effect failed with Got back unexpected response : Failed executing action ActionObject NumpyArrayObject[248..5].sort(,), result is an error: Failed executing action ActionObject NumpyArrayObject[248..5].sort(,), result is an error: '<' not supported between instances of 'NoneType' and 'str'
 Traceback (most recent call last):
  File "/Users/theresa/opt/anaconda3/envs/OpenMined/lib/python3.9/site-packages/syft/service/action/action_object.py", line 333, in send_action_side_effect
    raise RuntimeError(f"Got back unexpected response : {action_result}")
RuntimeError: Got back unexpected response : Failed executing action ActionObject NumpyArrayObject[248..5].sort(,), result is an error: Failed executing action ActionObject NumpyArrayObject[248..5].sort(,), result is an error: '<' not supported between instances of 'NoneType' and 'str'



```python
TwinPointer(Mock)
```
array(['10000', '10001', '10002', '10004', '10005', '10008', '10009',
       '10011', '10012', '10013', '10014', '10015', '10016', '10017',
       '10019', '10020', '10022', '10023', '10024', '10025', '10027',
       '10029', '10030', '10031', '10032', '10033', '10034', '10036',
       '10037', '10038', '10040', '10041', '10042', '10043', '10044',
       '10045', '10046', '10048', '10050', '10052', '10053', '10054',
       '10055', '10056', '10057', '10060', '10064', '10066', '10067',
       '10069', '10073', '10074', '10075', '10076', '10077', '10078',
       '10079', '10080', '10081', '10083', '10086', '10087', '10089',
       '10091', '10092', '10093', '10094', '10095', '10096', '10099',
       '10101', '10102', '10103', '10104', '10105', '10106', '10107',
       '10109', '10110', '10111', '10112', '10113', '10114', '10115',
       '10116', '10117', '10118', '10120', '10121', '10122', '10123',
       '10125', '10129', '10132', '10133', '10134', '10135', '10136',
       '10139', '10140', '10142', '10144', '10145', '10146', '10147',
       '10148', '10149', '10150', '10151', '10154', '10155', '10157',
       '10158', '10159', '10161', '10162', '10163', '10164', '10165',
       '10167', '10168', '10169', '10171', '10173', '10176', '10178',
       '10179', '10180', '10181', '10182', '10183', '10186', '10187',
       '10188', '10192', '10194', '10195', '10198', '10201', '10202',
       '10204', '10206', '10207', '10208', '10209', '10210', '10213',
       '10214', '10215', '10216', '10217', '10218', '10219', '10221',
       '10224', '10228', '10230', '10231', '10232', '10235', '10236',
       '10238', '10239', '10240', '10241', '10242', '10243', '10244',
       '10249', '10250', '10251', '10252', '10253', '10254', '10255',
       '10257', '10258', '10259', '10260', '10262', '10263', '10265',
       '10266', '10267', '10269', '10271', '10272', '10273', '10276',
       '10279', '10280', '10282', '10283', '10284', '10287', '10290',
       '10292', '10294', '10297', '10298', '10300', '10301', '10302',
       '10303', '10304', '10305', '10307', '10309', '10310', '10311',
       '10312', '10314', '10317', '10319', '10320', '10322', '10323',
       '10324', '10328', '10329', '10330', '10331', '10334', '10335',
       '10336', '10337', '10340', '10344', '10346', '10351', '10354',
       '10355', '10356', '10357', '10358', '10359', '10360', '10362',
       '10363', '10366', '10367', '10369', '10370', '10371', '10372',
       '10374', '10376', '10380', '10382', '10384', '10385', '10386',
       '10389', '10392', '10393', '10394', '10396', '10397', '10398',
       '10399', '10400', '10402', '10403', '10404', '10406', '10407',
       '10408', '10409', '10410', '10411', '10413', '10414', '10415',
       '10417', '10418', '10419', '10420', '10421', '10422', '10423',
       '10425', '10426', '10428', '10429', '10430', '10431', '10432',
       '10433', '10434', '10435', '10436', '10437', '10438', '10441',
       '10443', '10444', '10445', '10447', '10448', '10449', '10451',
       '10452', '10455', '10456', '10457', '10458', '10459', '10460',
       '10461', '10462', '10463', '10465', '10466', '10467', '10471',
       '10473', '10482', '10483', '10484', '10485', '10486', '10487',
       '10489', '10490', '10491', '10494', '10495', '10496', '10497',
       '10499', '10502', '10503', '10504', '10507', '10508', '10512',
       '10515', '10516', '10517', '10518', '10523', '10528', '10531',
       '10532', '10536', '10543', '10544', '10548', '10550', '10551',
       '10553', '10554', '10562', '10565', '10566', '10568', '10569',
       '10570', '10573', '10577', '10578', '10581', '10582', '10583',
       '10587', '10588', '10590', '10593', '10594', '10600', '10601',
       '10602', '10603', '10604', '10606', '10607', '10610', '10612',
       '10613', '10614', '10615', '10616', '10618', '10619', '10620',
       '10622', '10623', '10625', '10627', '10630', '10634', '10637',
       '10639', '10640', '10641', '10642', '10644', '10645', '10647',
       '10648', '10649', '10650', '10651', '10652', '10653', '10655',
       '10656', '10659', '10665', '10666', '10667', '10668', '10670',
       '10673', '10674', '10675', '10676', '10678', '10679', '10680',
       '10682', '10683', '10685', '10686', '10687', '10688', '10692',
       '10693', '10696', '10700', '10701', '10702', '10704', '10705',
       '10709', '10710', '10711', '10712', '10713', '10714', '10717',
       '10718', '10719', '10720', '10721', '10722', '10723', '10724',
       '10725', '10727', '10728', '10729', '10731', '10734', '10735',
       '10736', '10737', '10738', '10740', '10741', '10742', '10743',
       '10744', '10745', '10747', '10748', '10750', '10752', '10756',
       '10758', '10759', '10760', '10762', '10763', '10764', '10766',
       '10767', '10768', '10769', '10772', '10773', '10774', '10775',
       '10777', '10781', '10782', '10783', '10784', '10785', '10787',
       '10788', '10789', '10790', '10791', '10792', '10793', '10794',
       '10795', '10796', '10797', '10799', '10800', '10801', '10802',
       '10803', '10804', '10805', '10806', '10807', '10808', '10809',
       '10810', '10811', '10813', '10814', '10818', '10819', '10822',
       '10823', '10824', '10826', '10827', '10829', '10831', '10832',
       '10833', '10834', '10836', '10837', '10838', '10839', '10842',
       '10844', '10847', '10849', '10851', '10852', '10853', '10854',
       '10855', '10857', '10858', '10860', '10861', '10862', '10864',
       '10866', '10868', '10869', '10870', '10872', '10874', '10876',
       '10877', '10878', '10879', '10881', '10883', '10884', '10885',
       '10886', '10887', '10888', '10889', '10890', '10891', '10892',
       '10893', '10894', '10895', '10897', '10900', '10903', '10906',
       '10907', '10908', '10909', '10911', '10912', '10914', '10916',
       '10917', '10918', '10921', '10922', '10923', '10924', '10925',
       '10926', '10927', '10929', '10930', '10931', '10932', '10933',
       '10934', '10935', '10936', '10939', '10940', '10941', '10942',
       '10945', '10946', '10947', '10949', '10950', '10952', '10953',
       '10954', '10955', '10957', '10960', '10962', '10963', '10964',
       '10965', '10967', '10968', '10969', '10972', '10973', '10974',
       '10975', '10976', '10977', '10978', '10979', '10980', '10983',
       '10985', '10987', '10989', '10990', '10991', '10992', '10993',
       '10995', '10996', '10997', '10998'], dtype=object)

Amazing! This is much cleaner. There's something a bit weird here, though -- I looked up 77056 on Google maps, and that's in Texas.

Let's take a closer look:

In [29]:
zips = requests['Incident Zip']
# Let's say the zips starting with '0' and '1' are okay, for now. (this isn't actually true -- 13221 is in Syracuse, and why?)
is_close = zips.str.startswith('0') | zips.str.startswith('1')
# There are a bunch of NaNs, but we're not interested in them right now, so we'll say they're False
is_far = ~(is_close) & zips.notnull()

In [30]:
zips[is_far]

```python
TwinPointer(Mock)
```
Series([], Name: Incident Zip, dtype: object)

Okay, there really are requests coming from LA and Houston! Good to know. Filtering by zip code is probably a bad way to handle this -- we should really be looking at the city instead.



In [31]:
requests['City'].str.upper().value_counts()

```python
TwinPointer(Mock)
```
NEW YORK    351
BRONX       345
BROOKLYN    304
Name: City, dtype: int64

It looks like these are legitimate complaints, so we'll just leave them alone.

## Putting it together

In [32]:
unique_zips.request(guest_client)

```python
class Request:
  id: str = 067672b05ab24861a4da8c7a9ee88f36
  requesting_user_verify_key: str = 368729a48a43168350cb4e6eab9e82a9c3a32730591308c56e7897c28949eefb
  approving_user_verify_key: str = None
  request_time: str = 2023-05-30 17:04:11
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 281e55cb3425460e864682cfb4672fcb
  request_hash: str = "a2bf84d51f8bd87585406ee098247488ed4490259d295324b359abb9d0d0fbc1"
  changes: str = [syft.service.request.request.ActionStoreChange]

```

Request code execution

# Data owner: approve request

In [33]:
root_domain_client = node.login(email="info@openmined.org", password="changethis")

In [34]:
root_domain_client.api.services.request[0].approve()

<class 'syft.service.response.SyftSuccess'>: Request 067672b05ab24861a4da8c7a9ee88f36 changes applied

# Data scientist: fetch result

In [35]:
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

In [36]:
real_result = unique_zips.get_from(guest_client)

In [37]:
real_result

array(['11432', '11378', '10032', '10023', '10027', '11372', '11419',
       '11417', '10011', '11225', '11218', '10003', '10029', '10466',
       '11219', '10025', '10310', '11236', None, '10033', '11216',
       '10016', '10305', '10312', '10026', '10309', '10036', '11433',
       '11235', '11213', '11379', '11101', '10014', '11231', '11234',
       '10457', '10459', '10465', '11207', '10002', '10034', '11233',
       '10453', '10456', '10469', '11374', '11221', '11421', '11215',
       '10007', '10019', '11205', '11418', '11369', '11249', '10005',
       '10009', '11211', '11412', '10458', '11229', '10065', '10030',
       '11222', '10024', '10013', '11420', '11365', '10012', '11214',
       '11212', '10022', '11232', '11040', '11226', '10281', '11102',
       '11208', '10001', '10472', '11414', '11223', '10040', '11220',
       '11373', '11203', '11691', '11356', '10017', '10452', '10280',
       '11217', '10031', '11201', '11358', '10128', '11423', '10039',
       '10010', '11209'

In [38]:
node.land()